<a href="https://colab.research.google.com/github/deepskandpal/hands-on-machine-learning/blob/POC/language_Model_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Building an N-gram Language Model

In [5]:
import nltk
nltk.download('reuters')
nltk.download('punkt')
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

# Create a placeholder for model
model = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurance  
for sentence in reuters.sents():
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1
 
# Let's transform the counts to probabilities
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
dict(model["today", "the"])

{'Bank': 0.05555555555555555,
 'European': 0.05555555555555555,
 'Higher': 0.05555555555555555,
 'Italian': 0.05555555555555555,
 'Turkish': 0.05555555555555555,
 'company': 0.16666666666666666,
 'emirate': 0.05555555555555555,
 'increase': 0.05555555555555555,
 'newspaper': 0.05555555555555555,
 'options': 0.05555555555555555,
 'overseas': 0.05555555555555555,
 'pound': 0.05555555555555555,
 'price': 0.1111111111111111,
 'public': 0.05555555555555555,
 'time': 0.05555555555555555}